In [9]:
# Diverse functions for various ML tasks

import numpy as np
import math

# ********* AUXILIARY FUNCTIONS ********
# This avoids the errors that normally occur if you try to take the
# log of a non-positive value.
def safelog(x,base=2): return math.log(x,base) if x > 0 else 0

# Convert a list of numbers into a list wherein all numbers sum
# to 1 or, when geometric = True, to a list wherein the sum of
# the SQUARES of all numbers is 1.

def normalize_list(elems,geometric=False):
    if geometric:
        s = math.sqrt(sum(map((lambda x: x**2), elems)))
    else:
        s = sum(elems)
    if s != 0:
        return [elem / s for elem in elems]
    else:
        return n_of(len(elems),0)

# *************************************

# This takes in any list of NON-NEGATIVE numbers and a) converts it to
# a probability distribution, b) computes the entropy of that distribution.
# E.g. entropy([1,2,3,4]) => 1.846,  entropy([0.25]*4) => 2.0,
# entropy([0.125]*8) => 3.0, entropy([1]*8) => 3.0, etc.

def entropy(items, base=2):
    distribution = normalize_list(items)
    return -sum([d*safelog(d,base) for d in distribution])

# A master set has been partitioned, so compute entropy of each subset and
# then weight each entropy by the normalized size of the subset.  This is
# the main entropy calculation in algorithms such as ID3 and C4.5.
# NOTE:  The subsets should NOT be normalized; otherwise, each subset will
# have the same sum and size-normalization will be useless.
# E.g. For a 3-way partition into subsets representing two classes:
#  partition_entropy(((4,1),(4,2),(2,3))) => 0.873.
# For a 2-way partition into subsets representing 4 classes:
#  partition_entropy(((5,0,0,1),(2,2,2,2))) => 1.4214, and
# partition_entropy(((0,6,0,0),(8,0,0,0))) => 0.0
# In this last example, (0,6,0,0) represents a subset where all 6 items
# are in class 2, while none are in classes 1, 3 or 4.

def partition_entropy(partition):
    sizes = normalize_list([sum(sub) for sub in partition])
    sub_entropies = [entropy(sub) for sub in partition]
    return sum([s*e for s,e in zip(sizes,sub_entropies)])

# This computes the Standard Deviation Reduction (SDR) for different partitions of the examples at any
# node of a regression tree.  The classvals slot = values of the "class" for each case at the current node
# of a regression tree.

class RegTreeSplitter:

    def __init__(self,classvals):
        self.classvals = classvals # dict {case-id: class-value}
        self.size = len(classvals)
        cvals = list(self.classvals.values())
        self.avg = np.average(cvals)
        self.stdev = np.std(cvals)


    # Input = a subset of keys.  Output = stdev of classvals associated with those keys
    def subset_stdev(self,subset):
        vals = [self.classvals[key] for key in subset]
        if len(vals) > 0:
            return np.std(vals)
        else: return 0

    # partition = list of subsets (of the keys)
    def partition_stdev(self,partition):
        subset_stdevs = [self.subset_stdev(ss) for ss in partition]
        scaled_stdevs = [std*len(ss)/self.size for ss,std in zip(partition,subset_stdevs)]
        return subset_stdevs, scaled_stdevs, sum(scaled_stdevs)

    def stdev_reduction(self,partition):
        stdevs, scaled_stdevs, pstdev = self.partition_stdev(partition)
        sdr = self.stdev - pstdev
        print('Subset Stdevs = ', stdevs)
        print('Scaled Stdevs = ', scaled_stdevs)
        print('StDev Reduction = ', sdr)
        return sdr

def regtest():
    master = {'a':20.3 ,'b':20.0 ,'c':19.9 ,'d':20.3 ,'e':20.10 ,'f':20.15 ,'g': 20.0,'h': 19.80}
    rts = RegTreeSplitter(master)
    return rts

# This performs classic linear regression: it finds the closed-form solution (W) when mean-squared-error (MSE)
# is to be minimized.  Inputs = X (array of feature vectors, one per row), Y = target vector.
# This is minimizing MSE = (Y - WX)**2 by taking d(MSE) / dW and setting it to zero.  The closed-form
# solution (see my slides for linear-regression in AI-prog lectures (ML)) is:
#  W = Y * transpose(X) * (transpose(X) * X)**(-1)

def linreg(features,targets):
    num_features = features.shape[1]
    f2 = np.insert(features, 0, np.ones(features.shape[0]),axis=1)  # Add column of 1's to accommodate bias weight, w[0].
    f2_trans = np.transpose(f2)
    x2 = f2_trans @ f2  # NOte:  @ is shorthand for np.matmul
    #w = np.matmul(np.matmul(f2_trans,targets),np.linalg.inv(x2))
    w = np.matmul(np.linalg.inv(x2), np.matmul(f2_trans,targets))  # The formula given on Wikipedia
    print('Shapes: w = {}, f2 = {}'.format(w.shape,f2.shape))
    predictions = w @ f2_trans
    print('Predictions = {}'.format(predictions))
    return w, sum(np.square(targets - predictions))

def lint():
    x = np.array(range(1,11)).reshape(5, 2)  # 5 cases of size 2
    y = 11 + 3 * x[:, 0] + 5 * x[:, 1]  # weights = (11, 3, 5).  Linreg should FIND these !!
    return x,y, linreg(x,y)

In [41]:
entropy([1*8,2*8])

0.9182958340544896

In [21]:
partition_entropy(((2, 3),(2, 3),(4, 2)))

0.9512050593046015

In [25]:
partition_entropy(((5, 6),(3, 2)))

0.9868178311574916

In [27]:
partition_entropy(((5, 3),(3, 5)))

0.9544340029249649

In [55]:
partition_entropy(((1, 3, 2),(2, 3, 2, 3)))

1.7790245904193847

In [57]:
partition_entropy(((2, 2, 1),(2, 3, 1),(2, 1, 2)))

1.498385528189818